In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.chains.question_answering.chain import load_qa_chain

In [7]:
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPEN_AI_API_KEY"

In [9]:
# Model Loading (Embeddings & LLM)

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=200)

In [10]:
# Load PDF

pdf_link = "example.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [11]:
# Separate document in chunks

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 4000,
  chunk_overlap = 20,
  length_function = len,
  add_start_index = True,
)

chunks = text_splitter.split_documents(pages)

In [12]:
# Save chunks to vector store (Chroma database)

db = Chroma.from_documents(
  chunks, 
  embedding=embeddings_model, 
  persist_directory="text_index"
)

In [13]:
# Load database

vector_db = Chroma(
  persist_directory="text_index", 
  embedding_function=embeddings_model
)

# Load retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# Chain creation for LLM prompt
chain = load_qa_chain(llm, chain_type="stuff")

/var/folders/m7/3rgr80p16n9gk230pq2fkg9h0000gn/T/ipykernel_31067/2194551600.py:12: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [14]:
def ask(question):
  context = retriever.invoke(question)
  answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
  return answer, context

In [15]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)

/var/folders/m7/3rgr80p16n9gk230pq2fkg9h0000gn/T/ipykernel_31067/778930965.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']


Answer:  This PDF seems to be a comprehensive guide for beginners in audio mixing and processing. It covers topics like clipping correctly, mix bus compression, multi-band processing, controlling dynamics, using gain effectively, and addressing issues with loudness and laziness in mixes. Each section delves into different aspects of audio engineering, providing information, tips, and strategies to improve the quality of mixes.


In [16]:
print(context[0])

page_content='Section 01  The Beginners Guide To Clipping Correctly .................................................
 1. Stop Going Into The Red .......................................................................................
 2. So If Clipping Has Always Been Bad, Why Would I Want To Do It? ..................
	 3.	 What	Other	Benefits	Do	Peak	Clippers	Offer?	......................................................
 4. What If I Want To Use a Clipper For Mastering? .................................................
 5. Have your own clipping examples? ......................................................................
Section 02  The Beginner’s Guide to Mix Bus Compression ...........................................
 1. What is Mix Bus Compression? ............................................................................
 2. How Does Mix Bus Compression Affect My Mixes? ...........................................
 3. Common Mix Bus Compressors ...............................